# Preproccessing tasks

## User Chunk

User Chunk
* `customer_id`, `user_id`: Không có giá trị thiếu. Đây là các biến định danh, không mang ý nghĩa thống kê trong phân tích dữ liệu.
* `gender`: Không có giá trị `Null` và không có ngoại lệ. Dữ liệu bị mất cân bằng giữa các nhóm giới tính (`nam`, `nữ`, `khác`). Biến này cung cấp thông tin về giới tính người dùng và có thể ảnh hưởng đến hành vi khách hàng. Cần xem xét xử lý mất cân bằng dữ liệu trong các bước phân tích hoặc mô hình hoá.
* `location`, `province`, `region`: Không có giá trị `Null`. Các biến này mô tả vị trí địa lý của người dùng (`mã khu vực`, `tỉnh`, `vùng`). Dữ liệu phân bố không đồng đều giữa các giá trị, một số khu vực có rất ít người dùng. Cần gộp nhóm hoặc loại bỏ các khu vực có tần suất thấp để đảm bảo tính ổn định thống kê.
* `membership`: Không có giá trị `Null`. Đây là thông tin về hạng thành viên của người dùng, có ảnh hưởng đến mức độ trung thành và chiến lược marketing. Dữ liệu phân bố chênh lệch giữa các hạng, phần lớn khách hàng thuộc hạng `standard`, trong khi số lượng `diamond` rất ít. Cần cân nhắc xử lý sự mất cân bằng này.
* `sync_status_id`: Có `338,285` giá trị `Null`. Các giá trị còn lại đều bằng `2`, không có biến thiên nên không cung cấp nhiều thông tin cho phân tích.
* `location_name`: Không có giá trị `Null`. Dữ liệu có độ chi tiết cao và tần suất lặp lại thấp, dẫn đến phân bố rời rạc và thiếu tính khái quát. Biến này ít có giá trị cho các phân tích tổng quan.
* `install_app`: Không có giá trị `Null`. Dữ liệu phân bố không đều, trong đó các nhóm `In-Store`, `SPE`, `Android`, và `iOS` chiếm phần lớn. Các giá trị như `Wholesale`, `Chat`, `Call`, `LZD` và `Không xác định` chiếm tỉ lệ rất nhỏ. Có thể gộp các nhóm nhỏ thành `Khác` hoặc loại bỏ để tránh nhiễu.
* `district`: Không có giá trị `Null`. Dữ liệu có độ chi tiết cao và tần suất lặp lại thấp, nhưng khái quát hơn so với `location_name`. Có thể giữ lại nếu cần phân tích xu hướng khách hàng theo từng `province`.
* `is_deleted`: Tất cả giá trị đều là `False`, không cung cấp thông tin phân biệt và không có giá trị cho phân tích.

### EDA đa biến
* **`customer_id` - `timestamp`**, **`timestamp` - `install_date`** tương quan đều **rất cao** và R²=0.9756. Cần loại bỏ 1 trong 2 biến

* **`customer_id` - `location`**
Tương quan yếu (r≈0.16, R²≈0.026), nhưng vẫn có mối quan hệ nhẹ: khách hàng mới (ID cao) có xu hướng xuất hiện ở khu vực có mã `location` lớn hơn (có thể là tỉnh/thành mới mở rộng). Nên **giữ lại cả hai**, vì cung cấp góc nhìn khác (địa lý và thời gian).

* **`install_app` - `gender`**
Dữ liệu cho thấy khách hàng nữ chiếm ưu thế rõ rệt trên hầu hết kênh: In-Store ~2.797M vs Nam 1.144M (≈71% nữ), SPE 347,828 vs 80 (gần như toàn bộ là nữ), Android/iOS cũng tương tự (Android: 91,827 nữ vs 124 nam; iOS: 127,642 nữ vs 240 nam).

## Purchase Chunk

Purchase Chunk  

không cột nào có giá trị NULL

* `timestamp`: cho thấy phân bố khá đều và không có outlier. Dữ liệu hiện ở dạng UNIX timestamp nên cần chuyển sang định dạng ngày-giờ (datetime) để thuận tiện cho phân tích xu hướng theo thời gian. Biến này nên được giữ lại và không cần xử lý Null hay Outlier.
* `user_id`: Dữ liệu cho thấy một số người mua hàng rất thường xuyên (tối đa 893 lần), trong khi phần lớn chỉ xuất hiện ít hơn. Phân bố này bị **mất cân bằng mạnh và nhiều outlier**, tập trung vào một nhóm nhỏ khách hàng có hoạt động cao. Biến này có giá trị cho việc phân tích hành vi mua hàng và nhận diện nhóm khách hàng thân thiết, nhưng cần xử lí outlier
* `item_id`: một số sản phẩm có số lượt mua rất cao (đến hơn 300.000 lượt), trong khi phần lớn các sản phẩm khác được mua ít hơn nhiều, cho thấy **phân bố mất cân bằng mạnh**. Biến này quan trọng để phân tích độ phổ biến của sản phẩm và xu hướng tiêu dùng, nhưng cần xử lí outlier
* `event_type`: chỉ chứa một giá trị duy nhất là **“Purchase”** cho toàn bộ 35,729,825 bản ghi. Do không có sự biến thiên, biến này **không mang thông tin phân biệt** và không đóng góp giá trị cho các phân tích thống kê hoặc mô hình dự đoán. Vì vậy, **nên loại bỏ biến này** khỏi tập dữ liệu để tránh dư thừa.
* `event_value`: Giá trị trung bình là 1.51 và độ lệch chuẩn khá lớn, trong khi hầu hết các quan sát đều bằng 1. Boxplot cho thấy nhiều **outlier** với giá trị vượt xa phần lớn dữ liệu, tối đa lên đến 240. Biến này cần được **trimming hoặc winsorizing** trước khi đưa vào phân tích
* `price`: độ lệch chuẩn lớn **~195.000**, ức giá dao động mạnh từ **0.0444** đến **20.990.000**, xuất hiện nhiều giá trị cực lớn so với trung vị. Do đó, biến này có nhiều **outlier** và cần được **trimming hoặc winsorizing** trước khi đưa vào phân tích
* `date_key`: Biến date_key phân bố không đều theo thời gian, nên khi trực quan hóa hay huấn luyện mô hình, cần tránh giả định rằng dữ liệu này đại diện cho toàn bộ giai đoạn liên tục.
* `quantity`: tập trung chủ yếu ở giá trị 1. Các giá trị cao (ví dụ >50) có thể là đơn hàng sỉ hoặc dữ liệu bất thường, nên khi phân tích thống kê hoặc huấn luyện mô hình, cần xem xét rimming hoặc log-transform để giảm ảnh hưởng của các ngoại lệ này.
* `customer_id`:  đa số khách hàng chỉ mua vài lần (không được thể hiện ở top 10), còn một số ít khách hàng mua với tần suất rất cao. Nên xem xét tách nhóm khách hàng hoạt động mạnh để phân tích riêng, vì họ có thể ảnh hưởng lớn đến trung bình toàn tập dữ liệu.
* `created_date`: Các giá trị phân bố tương đối đều theo thời gian, trung vị rơi vào **02/07/2024**, cho thấy hoạt động diễn ra ổn định trong năm, **không có dấu hiệu mất cân đối** rõ rệt giữa các giai đoạn.
* `payment`: có nhóm giá trị không xác định (331952). Dữ liệu **phân bố  không đều**, tập trung chủ yếu ở “Tiền mặt”, trong khi các phương thức khác chiếm tỷ lệ nhỏ. Không xuất hiện **ngoại lệ rõ rệt**, tuy nhiên cần **xử lý mất cân đối** khi phân tích hành vi hoặc huấn luyện mô hình.
* `location`: Phân bố **khá đồng đều và cân đối**, không xuất hiện ngoại lệ trên boxplot. Không cần xử lý **Null** hay **Outlier**, và cũng **không có dấu hiệu phân bố lệch**, nên có thể giữ nguyên để dùng trong phân tích.
* `discount`: phần lớn giá trị bằng **0**, trong khi một số ít đạt tới **7.995e6**, cho thấy **nhiều outlier**. Cần **chuẩn hóa phân bố** để tránh ảnh hưởng đến kết quả thống kê.
* `is_deleted`: Tất cả giá trị đều là **False**. Biến này **không mang thông tin phân biệt**, nên **nên loại bỏ** khỏi phân tích


### EDA đa biến  
* **`event_value` - `quantity`**
Hai biến này có **tương quan hoàn hảo (r = 1.0)**. Việc giữ cả hai gây **multicollinearity**, nên chỉ nên giữ một biến — thường là `quantity` để phản ánh số lượng thực tế.

* **`quantity` - `discount`**
Có **tương quan dương yếu (r ≈ 0.17)**, cho thấy khi số lượng mua tăng, mức giảm giá trung bình cũng tăng. Điều này phản ánh **chính sách khuyến mãi theo số lượng**, phù hợp với logic kinh doanh. Có thể giữ lại cả hai biến, nhưng biến `discount` nên được chuẩn hóa do phân bố lệch.

* **`price` - `discount`**, **`event_value` - `discount`**
Tương quan **dương nhẹ**. Nên giữ cả hai biến để **phân tích chính sách giá và khuyến mãi**, và **mô hình dự đoán hành vi khuyến mãi**.

* **`timestamp` - `date_key`**
Hai biến này có **tương quan gần tuyệt đối (r ≈ 0.998)**, vì cùng biểu diễn thông tin thời gian. `timestamp` mang tính chính xác hơn nên **nên giữ lại biến này** và **loại `date_key`** để tránh dư thừa dữ liệu.

* **`customer_id` - `location`**
Có **tương quan yếu (r ≈ 0.10)**, cho thấy khách hàng ở một số khu vực có xu hướng mua sắm nhiều hơn. Mối quan hệ này có thể hữu ích để **phân tích hành vi theo vùng địa lý**.

* **`channel` - `price`**
Sự khác biệt về giá trung bình giữa các kênh là rõ rệt và có ý nghĩa thống kê cao. Các kênh như Call, Chat, CRM Partner, SPE có giá trung bình cao hơn (≈300K-350K) so với In-Store (≈160K).

* **`item_id` - `discount`**
Chiết khấu khác biệt mạnh giữa các mặt hàng. Kết quả cho thấy chính sách khuyến mãi được áp dụng chọn lọc theo sản phẩm, phản ánh chiến lược ưu tiên giải phóng hàng tồn hoặc đẩy doanh số từng nhóm cụ thể.

* **`item_id` - `price`**
Giá bán trung bình khác biệt rất lớn giữa các sản phẩm, từ khoảng 31K đến 774K, với độ lệch chuẩn cao và phân bố lệch phải. Sự khác biệt này chủ yếu đến từ đặc trưng sản phẩm (phân khúc giá, loại hàng) hơn là biến động ngẫu nhiên.

* **`channel` - `payment`**
Giá trị Chi-square đạt **24,121,483.6** với **p = 0.0000**, thể hiện **mối liên hệ cực kỳ mạnh và có ý nghĩa thống kê cao** giữa kênh bán hàng và phương thức thanh toán.


## Item Chunk

Item Chunk  
Tổng quan dữ liệu
* **Số dòng:** 27.332
* **Số cột:** 34
* **Không có bản ghi trùng lặp (`item_id` unique = 27.332)**
* **Nhiều cột có giá trị "Không xác định" chiếm phần lớn**
* **Tất cả sản phẩm đều `is_deleted = false`**
* `item_id`: Tất cả giá trị đều **duy nhất** và không có giá trị Null. Đây là khóa định danh chính cho từng sản phẩm. không cần giữ lại vì chỉ mang tính định danh, không cung cấp thông tin thống kê
* `is_deleted`: Tất cả sản phẩm có giá trị **False**. Biến này **không mang thông tin phân biệt** và có thể loại bỏ
* `created_date`: Các giá trị phân bố tương đối đều theo thời gian
* `price`: Biến này có nhiều **outlier** và cần được **trimming hoặc winsorizing** trước khi đưa vào mô hình.
* `gp`: Nhiều giá trị nhỏ hoặc bằng 0, thể hiện phân phối lệch phải. Nên kiểm tra tỷ lệ lợi nhuận âm hoặc 0 để xác định sản phẩm bán lỗ hoặc không có dữ liệu chính xác.
* `weight`: Có nhiều giá trị bằng **0 hoặc không xác định**. Cần chuẩn hóa đơn vị đo trước khi phân tích tương quan với giá hoặc lợi nhuận.
* `category_l1`: Phân phối mất cân đối, thiên về nhóm thời trang trẻ em. Biến này hữu ích để phân tích xu hướng ngành hàng nhưng cần xử lý mất cân đối khi huấn luyện mô hình.
* `category_l2`: Cấu trúc danh mục có dấu hiệu trùng lặp giữa các nhánh, nên xem xét tái phân loại hoặc gộp nhóm.
* `category_l3`: Nhiều giá trị lặp và phân bố không đều. Một số nhóm rất hiếm, có thể gây nhiễu khi one-hot encode. Cần gộp nhóm hoặc sử dụng kỹ thuật xử lý danh mục hiếm (rare category encoding).
* `brand`: Dữ liệu bị **thiếu nhãn thương hiệu** cho nhiều sản phẩm (70% là giá trị không xác định), cần làm sạch hoặc loại bỏ biến này.
* `manufacturer`: Hơn **94% giá trị “Không xác định”**, thể hiện mức độ thiếu dữ liệu nghiêm trọng. Biến này gần như không có giá trị phân biệt trong giai đoạn hiện tại, có thể loại bỏ.
* `gender_target`: 66% giá trị là **“Không xác định”**, phần còn lại chia nhỏ cho “Bé gái”, “Bé trai”, “Sơ sinh”. Dữ liệu bị mất cân đối mạnh và thiếu dữ liệu, cần thêm dữ liệu hoặc loại bỏ.
* `age_group`: 58% “Không xác định”, còn lại tập trung ở nhóm **0-3 tuổi**. Phân phối này cho thấy dữ liệu hướng đến sản phẩm trẻ nhỏ

### EDA đa biến
* Tương quan nổi bật duy nhất: Giữa price và gp (r = 0.8867).
→ Điều này cho thấy gp (gross profit) gần như tỷ lệ thuận với price, phù hợp với logic kinh doanh.

* Các kiểm định giữa item_id và các biến khác: Không có khác biệt đáng kể, nguyên nhân chủ yếu là cỡ mẫu cực nhỏ (mỗi nhóm chỉ 1 bản ghi).


* **`category_l2` - `sale_status`**
Biến `sale_status` có sự khác biệt rõ rệt giữa các nhóm `category_l2`. Các nhóm thời trang như **Thời trang bé trai**, **Modal kháng khuẩn**, **Thời trang bé gái** có tỷ lệ bán cao nhất (~27%), trong khi **Sách & VPP** và **Hàng cũ** gần như không bán được. Điều này cho thấy mức độ tiêu thụ phụ thuộc mạnh vào nhóm sản phẩm, phản ánh sự khác biệt về nhu cầu thị trường.

* **`category_l3` - `price`**
Có sự khác biệt rất lớn giữa các nhóm `category_l3` **Bộ Modal**, **Bộ bé trai**, **Bộ bé gái** có giá trung bình cao nhất (179–255K), trong khi **hàng sơ sinh cũ** và **sách** có giá thấp . Phân bố giá này thể hiện rõ cấu trúc phân khúc sản phẩm: nhóm “bộ” là hàng cao cấp, nhóm “cũ” là hàng giá rẻ.

* **`category_l3` - `creation_timestamp`**
Các nhóm có sự khác biệt đáng kể về thời gian tạo . **Bộ bé trai, bé gái, Modal** là nhóm mới được tạo gần đây, trong khi **hàng cũ** và **hàng sơ sinh cũ** có timestamp thấp hơn nhiều. Điều này phản ánh vòng đời sản phẩm và giúp nhận diện các nhóm hàng lỗi thời cần thanh lý.

* **`category_l3` - `gp` (gross profit)**
Khác biệt có ý nghĩa thống kê. Lợi nhuận gộp trung bình cao nhất thuộc về **Bộ bé gái (73K)** và **Bộ bé trai (63K)**, thấp nhất là **hàng cũ (5-8K)**. Phân tích này khẳng định các nhóm “bộ” không chỉ có giá cao mà còn mang lại lợi nhuận lớn, là nhóm sản phẩm chiến lược cần ưu tiên duy trì.

* **`category_l3` - `p_id`**
Có sự khác biệt rõ. Các nhóm “bộ” có giá trị `p_id` cao hơn (mã mới), còn các nhóm “hàng cũ” có `p_id` thấp. Đây là bằng chứng cho thấy vòng đời sản phẩm liên quan đến thế hệ mã hàng, giúp nhận diện hàng lỗi thời trong quản lý tồn kho.
